# Metadata

**L1 Taxonomy** - Backend Integration

**L2 Taxonomy** - API Integration

**Subtopic** - Using GraphQL introspection from Python to adapt to API schema changes

**Use Case** - Develop a Python module that uses GraphQL introspection to fetch the schema of a GraphQL API and adapt to any changes in the schema. The module should be able to handle changes in the API schema such as addition or removal of fields, changes in data types, and deprecation of fields. The module should use only standard Python libraries and packages from PyPI, and should not require any external infrastructure.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
requests==2.32.4
urllib3==2.5.0
```


# Prompt
## Problem Description
You have been asked to build a *self-adapting* Python module that can talk to any GraphQL server whose schema may evolve at runtime.  
The module must:

1. Issue the standard GraphQL introspection query to fetch the complete schema as JSON.  
2. Parse that schema and hold an in-memory representation of all object types, fields, arguments and their nullability.  
3. Generate and execute dynamic read queries that always include every scalar field of a chosen object type, even when the server adds or removes fields later.  
4. Detect server-side schema changes and refresh its internal model without restarting the process.  
5. Offer a minimal public API so callers can request  
   "give me all scalar fields of type *X* where *id* = *Y*" without writing GraphQL by hand.

The deliverable is a single importable file named main.py. Importing the module must perform *zero* network or file I/O; all side effects happen only inside public functions.

## Input Format
Caller code passes:

* endpoint_url – HTTPS GraphQL endpoint  
* optional headers – dict of extra HTTP headers (auth tokens etc.)  
* object_type – a type name found in the schema  
* filter – dict such as {"id": 42}  

Your module then:

1. POSTS the introspection query (unless a fresh schema is already cached in memory).  
2. Builds the read query string in deterministic field order.  
3. Executes the query and returns the JSON *data* section.

## Output Format
* On success return a native Python dict equal to response["data"].  
* On any error raise GraphQLAdaptiveError.  
* The module must never print to stdout or stderr except in the example below.

## Example
```python
from main import AdaptiveClient, GraphQLAdaptiveError

client = AdaptiveClient(
    endpoint="https://api.example.com/graphql",
    headers={"Authorization": "Bearer abc123"},
    refresh_interval_seconds=300      # auto refresh schema every 5 min
)

try:
    product = client.fetch_by_id(object_type="Product", object_id=17)
    print(product)   # {'id': 17, 'name': 'Widget', 'price': 12.5, ...}
except GraphQLAdaptiveError as err:
    print("GraphQL call failed:", err)
```

# Requirements
## Explicit Requirements
* Libraries – only Python 3.8 standard library plus the third-party package requests. No other dependencies.
* Class design
  * class AdaptiveClient with constructor  
    __init__(endpoint: str, headers: dict | None = None, refresh_interval_seconds: int = 0)
  * Public methods  
    * refresh_schema(force: bool = False) -> None  
      Perform introspection if the refresh interval has expired or force is True.  
    * fetch_by_id(object_type: str, object_id: int | str) -> dict  
      Build a query selecting every scalar field (and nested scalars of immediate non-nullable object fields) and return one record.  
    * run_raw(query: str, variables: dict | None = None) -> dict – low-level escape hatch.
* Schema cache
  * Store the last introspection result plus a timestamp in memory only.  
  * Respect refresh_interval_seconds; zero disables automatic refresh.  
  * Compare SHA-256 of the raw introspection JSON; rebuild internal structures if the hash changes.
* Dynamic query generation
  * Scalars include built-ins Int, Float, String, Boolean, ID plus custom scalars that map to JSON primitives.  
  * Ignore fields that require arguments other than id.  
  * Keep field order alphabetical so that generated query strings are deterministic.
* Networking
  * Use a single requests.Session with keep-alive per AdaptiveClient instance.  
  * Add Content-Type: application/json automatically.  
  * Follow up to three HTTP redirects while preserving headers.  
  * On status 429 or 503 back off and retry up to three times with delays 1 sec, 2 sec, 4 sec.
* Error handling
  * Define class GraphQLAdaptiveError(Exception).  
  * Raise on network failure, top-level "errors" from GraphQL, unknown object_type, or schema hash mismatch after a refresh.
* Performance
  * A single schema refresh (up to 1 MB JSON) must complete within 5 seconds.  
  * fetch_by_id must build its query in O(n) where n is the number of fields in the type.
* Logging
  * Use logging at INFO.  
  * Log every schema refresh, generated query string, variables, and total latency in milliseconds.
* File layout
  * Single file main.py, no more than 400 logical lines (comments excluded).  
  * Importing must not trigger network access, disk reads, or logging.

## Implicit Requirements
* HTTPS only; abort if the endpoint URL is not https://.
* Deterministic behavior – identical inputs yield byte-for-byte identical GraphQL queries.
* Respect caller-supplied headers on every request.
* Use json.dumps(separators=(",", ":")) for deterministic payloads.
* All timestamps in logs are UTC ISO-8601 with trailing Z.
* Unit-test friendly: if environment variable ADAPTIVE_GQL_OFFLINE=1 is set, skip network calls and raise GraphQLAdaptiveError.

## Edge Cases
* Server drops a custom scalar – treat as String without failing.
* Field removed between refreshes – detect on next access, raise GraphQLAdaptiveError.
* Introspection disabled on the server – raise with clear message.
* Network throttled to 50 KiB per second – still meet time limits.
* Enforce peak resident set size under 200 MB.

## Constraints
* No interactive prompts or input() calls.
* Works unchanged on CPython 3.8 through 3.12.
* Never write to disk or create temp files.

## Function and Class Signatures
```python
class GraphQLAdaptiveError(Exception): ...

class AdaptiveClient:
    def __init__(self,
                 endpoint: str,
                 headers: dict | None = None,
                 refresh_interval_seconds: int = 0): ...

    def refresh_schema(self, force: bool = False) -> None: ...

    def fetch_by_id(self,
                    object_type: str,
                    object_id: int | str) -> dict: ...

    def run_raw(self,
                query: str,
                variables: dict | None = None) -> dict: ...
```

## Important Notes

The grading harness may:

* Mock the endpoint to return evolving schemas with fields reordered.
* Inject random network delays and bandwidth limits.
* Use the resource module to check peak RSS.

Your implementation should handle all of these without timeouts, memory leaks, or nondeterministic output.


In [ ]:
# code
import os
import json
import time
import hashlib
import logging
from datetime import datetime, timezone
from typing import Set, Union

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Configure logging
logger = logging.getLogger(__name__)


class GraphQLAdaptiveError(Exception):
    """Raised for any GraphQL adaptive client error."""
    pass


class AdaptiveClient:
    """Self-adapting GraphQL client that handles evolving schemas."""

    INTROSPECTION_QUERY = """
    query IntrospectionQuery {
      __schema {
        queryType { name }
        mutationType { name }
        subscriptionType { name }
        types {
          kind
          name
          description
          fields(includeDeprecated: true) {
            name
            description
            args {
              name
              type { ...TypeRef }
              defaultValue
            }
            type { ...TypeRef }
            isDeprecated
            deprecationReason
          }
          inputFields {
            name
            description
            type { ...TypeRef }
            defaultValue
          }
          interfaces { ...TypeRef }
          enumValues(includeDeprecated: true) {
            name
            description
            isDeprecated
            deprecationReason
          }
          possibleTypes { ...TypeRef }
        }
        directives {
          name
          description
          locations
          args {
            name
            description
            type { ...TypeRef }
            defaultValue
          }
        }
      }
    }

    fragment TypeRef on __Type {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
            ofType {
              kind
              name
              ofType {
                kind
                name
                ofType {
                  kind
                  name
                  ofType {
                    kind
                    name
                  }
                }
              }
            }
          }
        }
      }
    }
    """

    SCALAR_TYPES = {
        'Int', 'Float', 'String', 'Boolean', 'ID'
    }

    def __init__(self,
                 endpoint: str,
                 headers: dict | None = None,
                 refresh_interval_seconds: int = 0):
        """Initialize the adaptive GraphQL client."""
        if not endpoint.startswith('https://'):
            raise GraphQLAdaptiveError("Endpoint must use HTTPS")

        self._endpoint = endpoint
        self._headers = headers or {}
        self._refresh_interval = refresh_interval_seconds

        # Schema cache
        self._schema_raw = None
        self._schema_hash = None
        self._schema_timestamp = 0
        self._schema_types = {}

        # Setup session with retry strategy
        self._session = self._create_session()

    def _create_session(self) -> requests.Session:
        """Create requests session with retry logic and proper configuration."""
        session = requests.Session()

        # Configure retry strategy for 429 and 503 only
        retry_strategy = Retry(
            total=3,
            status_forcelist=[429, 503],
            backoff_factor=1,  # 1s, 2s, 4s progression
            raise_on_status=False
        )

        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount('https://', adapter)

        # Set default headers
        session.headers.update({
            'Content-Type': 'application/json',
            **self._headers
        })

        return session

    def _check_offline_mode(self) -> None:
        """Check if offline mode is enabled via environment variable."""
        if os.environ.get('ADAPTIVE_GQL_OFFLINE') == '1':
            raise GraphQLAdaptiveError(
                "Offline mode enabled - network calls disabled")

    def _should_refresh_schema(self, force: bool) -> bool:
        """Determine if schema should be refreshed."""
        if force:
            return True

        if self._schema_raw is None:
            return True

        if self._refresh_interval == 0:
            return False

        elapsed = time.time() - self._schema_timestamp
        return elapsed >= self._refresh_interval

    def _execute_request(self, payload: dict) -> dict:
        """Execute GraphQL request with proper error handling."""
        self._check_offline_mode()

        start_time = time.time()

        try:
            response = self._session.post(
                self._endpoint,
                json=payload,
                timeout=5.0
            )
            response.raise_for_status()

            result = response.json()

            # Check for GraphQL errors
            if 'errors' in result:
                error_msg = '; '.join(err.get('message', 'Unknown error')
                                      for err in result['errors'])
                raise GraphQLAdaptiveError(f"GraphQL errors: {error_msg}")

            if 'data' not in result:
                raise GraphQLAdaptiveError("No data in GraphQL response")

            return result

        except requests.exceptions.RequestException as e:
            raise GraphQLAdaptiveError(f"Network error: {e}")
        except json.JSONDecodeError as e:
            raise GraphQLAdaptiveError(f"Invalid JSON response: {e}")
        finally:
            duration_ms = (time.time() - start_time) * 1000
            logger.info(f"GraphQL request completed in {duration_ms:.2f}ms")

    def _compute_schema_hash(self, schema_data: dict) -> str:
        """Compute SHA-256 hash of schema data."""
        schema_json = json.dumps(
            schema_data, separators=(',', ':'), sort_keys=True)
        return hashlib.sha256(schema_json.encode('utf-8')).hexdigest()

    def _parse_type_ref(self, type_ref: dict) -> tuple[str, bool]:
        """Parse GraphQL type reference to get type name and nullability."""
        if type_ref['kind'] == 'NON_NULL':
            inner_type, _ = self._parse_type_ref(type_ref['ofType'])
            return inner_type, False  # Non-nullable
        elif type_ref['kind'] == 'LIST':
            inner_type, nullable = self._parse_type_ref(type_ref['ofType'])
            return f"[{inner_type}]", True  # Lists are nullable by default
        else:
            return type_ref['name'], True  # Nullable

    def _is_scalar_type(self, type_name: str) -> bool:
        """Check if a type is a scalar type."""
        if type_name in self.SCALAR_TYPES:
            return True

        # Check if it's a custom scalar
        type_info = self._schema_types.get(type_name, {})
        return type_info.get('kind') == 'SCALAR'

    def _parse_schema(self, schema_data: dict) -> None:
        """Parse introspection schema into internal representation."""
        self._schema_types = {}

        for type_def in schema_data['__schema']['types']:
            type_name = type_def['name']

            # Skip GraphQL internal types
            if type_name.startswith('__'):
                continue

            parsed_type = {
                'kind': type_def['kind'],
                'name': type_name,
                'fields': {}
            }

            # Parse fields for OBJECT and INTERFACE types
            if type_def['kind'] in ('OBJECT', 'INTERFACE') and type_def.get('fields'):
                for field in type_def['fields']:
                    field_name = field['name']
                    field_type, nullable = self._parse_type_ref(field['type'])

                    # Check if field requires arguments (other than id)
                    args = field.get('args', [])
                    requires_args = any(arg['name'] != 'id' for arg in args)

                    parsed_type['fields'][field_name] = {
                        'type': field_type,
                        'nullable': nullable,
                        'requires_args': requires_args
                    }

            self._schema_types[type_name] = parsed_type

    def refresh_schema(self, force: bool = False) -> None:
        """Refresh schema from GraphQL endpoint."""
        if not self._should_refresh_schema(force):
            return

        start_time = time.time()
        timestamp_str = datetime.now(
            timezone.utc).strftime('%Y-%m-%dT%H:%M:%S.%fZ')

        logger.info(f"Starting schema refresh at {timestamp_str}")

        payload = {
            'query': self.INTROSPECTION_QUERY,
            'variables': {}
        }

        try:
            result = self._execute_request(payload)
            schema_data = result['data']

            # Check if schema actually changed
            new_hash = self._compute_schema_hash(schema_data)

            if self._schema_hash and self._schema_hash != new_hash:
                logger.info(
                    "Schema hash changed - rebuilding internal structures")
            elif self._schema_hash == new_hash:
                logger.info("Schema unchanged - using cached structures")
                self._schema_timestamp = time.time()
                return

            # Parse and store new schema
            self._parse_schema(schema_data)
            self._schema_raw = schema_data
            self._schema_hash = new_hash
            self._schema_timestamp = time.time()

            duration_ms = (time.time() - start_time) * 1000
            logger.info(f"Schema refresh completed in {duration_ms:.2f}ms")

        except Exception as e:
            if isinstance(e, GraphQLAdaptiveError):
                if "Introspection" in str(e):
                    raise GraphQLAdaptiveError(
                        "GraphQL introspection is disabled on this server")
                raise
            raise GraphQLAdaptiveError(f"Schema refresh failed: {e}")

    def _build_scalar_query(self, object_type: str, visited: Set[str] = None) -> str:
        """Build query string for all scalar fields of an object type."""
        if visited is None:
            visited = set()

        if object_type in visited:
            return ""  # Prevent infinite recursion

        visited.add(object_type)

        type_info = self._schema_types.get(object_type)
        if not type_info:
            raise GraphQLAdaptiveError(f"Unknown object type: {object_type}")

        if type_info['kind'] not in ('OBJECT', 'INTERFACE'):
            raise GraphQLAdaptiveError(f"Type {object_type} is not queryable")

        fields = []

        # Get all fields in alphabetical order for deterministic output
        for field_name in sorted(type_info['fields'].keys()):
            field_info = type_info['fields'][field_name]

            # Skip fields that require arguments (except id)
            if field_info['requires_args']:
                continue

            field_type = field_info['type']

            # Handle list types
            if field_type.startswith('[') and field_type.endswith(']'):
                inner_type = field_type[1:-1]
                if self._is_scalar_type(inner_type):
                    fields.append(field_name)
            elif self._is_scalar_type(field_type):
                # Direct scalar field
                fields.append(field_name)
            elif not field_info['nullable']:
                # Non-nullable object field - include nested scalars
                try:
                    nested_query = self._build_scalar_query(
                        field_type, visited.copy())
                    if nested_query:
                        fields.append(f"{field_name} {{ {nested_query} }}")
                except GraphQLAdaptiveError:
                    # Skip if nested type not found or not queryable
                    pass

        return ' '.join(fields)

    def fetch_by_id(self, object_type: str, object_id: Union[int, str]) -> dict:
        """Fetch object by ID with all scalar fields."""
        # Ensure schema is fresh
        self.refresh_schema()

        # Build the query
        scalar_fields = self._build_scalar_query(object_type)
        if not scalar_fields:
            raise GraphQLAdaptiveError(
                f"No queryable scalar fields found for type {object_type}")

        query = f"""
        query FetchById($id: ID!) {{
          {object_type.lower()}(id: $id) {{
            {scalar_fields}
          }}
        }}
        """.strip()

        variables = {'id': str(object_id)}

        # Log the generated query
        query_compact = ' '.join(query.split())
        logger.info(f"Generated query: {query_compact}")
        logger.info(
            f"Variables: {json.dumps(variables, separators=(',', ':'))}")

        result = self.run_raw(query, variables)

        # Extract the specific object from response
        data = result.get('data', {})
        object_data = data.get(object_type.lower())

        if object_data is None:
            raise GraphQLAdaptiveError(
                f"No {object_type} found with id {object_id}")

        return object_data

    def run_raw(self, query: str, variables: dict | None = None) -> dict:
        """Execute raw GraphQL query."""
        payload = {
            'query': query,
            'variables': variables or {}
        }

        # Use deterministic JSON serialization
        payload_json = json.dumps(
            payload, separators=(',', ':'), sort_keys=True)
        logger.info(f"Executing query with payload: {payload_json}")

        return self._execute_request(payload)


In [ ]:
# tests
"""Unit tests for the AdaptiveClient class interact with a GraphQL endpoint."""

import unittest
from unittest.mock import patch
from main import AdaptiveClient, GraphQLAdaptiveError
import os
import time


class TestAdaptiveClient(unittest.TestCase):
    """Test cases for the AdaptiveClient class."""

    def setUp(self):
        """Set up an AdaptiveClient instance for testing."""
        self.endpoint = "https://example.com/graphql"
        self.client = AdaptiveClient(endpoint=self.endpoint)

    def test_invalid_endpoint_scheme(self):
        """Test that an insecure HTTP endpoint raises an error."""
        with self.assertRaises(GraphQLAdaptiveError):
            AdaptiveClient("http://insecure.com/graphql")

    def test_session_headers_applied(self):
        """Test that custom headers are applied to the session."""
        client = AdaptiveClient(self.endpoint, headers={"X-Test": "1"})
        self.assertEqual(client._session.headers["X-Test"], "1")
        self.assertEqual(client._session.headers["Content-Type"],
                         "application/json")

    def test_offline_mode_detection(self):
        """Test that offline mode is respected via environment variable."""
        os.environ["ADAPTIVE_GQL_OFFLINE"] = "1"
        with self.assertRaises(GraphQLAdaptiveError):
            self.client._check_offline_mode()
        os.environ["ADAPTIVE_GQL_OFFLINE"] = "0"

    def test_should_refresh_schema_first_time(self):
        """Test that schema refresh is needed when none is loaded yet."""
        self.assertTrue(self.client._should_refresh_schema(force=False))

    def test_should_refresh_schema_force(self):
        """Test forced schema refresh."""
        self.client._schema_raw = {"dummy": True}
        self.assertTrue(self.client._should_refresh_schema(force=True))

    def test_should_refresh_schema_interval_elapsed(self):
        """Test schema refresh when interval has elapsed."""
        self.client._refresh_interval = 1
        self.client._schema_raw = {"x": "y"}
        self.client._schema_timestamp = time.time() - 2
        self.assertTrue(self.client._should_refresh_schema(force=False))

    def test_should_not_refresh_schema_when_cached(self):
        """Test no schema refresh when data is fresh and cached."""
        self.client._refresh_interval = 10
        self.client._schema_raw = {"x": "y"}
        self.client._schema_timestamp = time.time()
        self.assertFalse(self.client._should_refresh_schema(force=False))

    def test_compute_schema_hash_consistency(self):
        """Test that hashing the same schema twice gives the same hash."""
        schema = {"types": [{"name": "A"}]}
        h1 = self.client._compute_schema_hash(schema)
        h2 = self.client._compute_schema_hash(schema)
        self.assertEqual(h1, h2)

    def test_parse_type_ref_scalar(self):
        """Test parsing of scalar type reference."""
        ref = {"kind": "SCALAR", "name": "String"}
        name, nullable = self.client._parse_type_ref(ref)
        self.assertEqual(name, "String")
        self.assertTrue(nullable)

    def test_parse_type_ref_non_null(self):
        """Test parsing of non-null wrapped scalar reference."""
        ref = {"kind": "NON_NULL", "ofType": {"kind": "SCALAR", "name": "Int"}}
        name, nullable = self.client._parse_type_ref(ref)
        self.assertEqual(name, "Int")
        self.assertFalse(nullable)

    def test_parse_type_ref_list(self):
        """Test parsing of list type reference."""
        ref = {"kind": "LIST", "ofType": {"kind": "SCALAR", "name": "String"}}
        name, nullable = self.client._parse_type_ref(ref)
        self.assertEqual(name, "[String]")
        self.assertTrue(nullable)

    def test_is_scalar_builtin(self):
        """Test detection of built-in scalar type."""
        self.assertTrue(self.client._is_scalar_type("Int"))

    def test_is_scalar_custom_scalar(self):
        """Test detection of user-defined scalar type."""
        self.client._schema_types["CustomScalar"] = {"kind": "SCALAR"}
        self.assertTrue(self.client._is_scalar_type("CustomScalar"))

    def test_is_not_scalar(self):
        """Test detection of non-scalar type."""
        self.client._schema_types["Obj"] = {"kind": "OBJECT"}
        self.assertFalse(self.client._is_scalar_type("Obj"))

    @patch("main.AdaptiveClient._execute_request")
    def test_refresh_schema_success(self, mock_exec):
        """Test successful schema refresh updates internal type registry."""
        mock_exec.return_value = {
            "data": {
                "__schema": {
                    "types": [
                        {
                            "kind": "OBJECT",
                            "name": "User",
                            "fields": [
                                {
                                    "name": "id",
                                    "type": {"kind": "SCALAR", "name": "ID"},
                                    "args": [],
                                }
                            ]
                        }
                    ]
                }
            }
        }
        self.client.refresh_schema(force=True)
        self.assertIn("User", self.client._schema_types)

    @patch("main.AdaptiveClient._execute_request")
    def test_refresh_schema_skips_if_same_hash(self, mock_exec):
        """Test schema is not reloaded if hash matches existing schema."""
        mock_schema = {
            "__schema": {
                "types": [
                    {"kind": "OBJECT", "name": "X", "fields": []}
                ]
            }
        }
        mock_exec.return_value = {"data": mock_schema}
        h = self.client._compute_schema_hash(mock_schema)
        self.client._schema_hash = h
        self.client._schema_raw = mock_schema
        self.client._schema_timestamp = time.time() - 100
        self.client._refresh_interval = 1
        self.client.refresh_schema()
        self.assertEqual(self.client._schema_hash, h)

    def test_build_scalar_query_basic(self):
        """Test scalar query generation for basic object."""
        self.client._schema_types["User"] = {
            "kind": "OBJECT",
            "fields": {
                "id": {"type": "ID", "nullable": True, "requires_args": False},
                "email": {"type": "String",
                          "nullable": True, "requires_args": False}
            }
        }
        result = self.client._build_scalar_query("User")
        self.assertIn("id", result)
        self.assertIn("email", result)

    def test_build_scalar_query_recursion_block(self):
        """Test that recursion is prevented when generating scalar queries."""
        self.client._schema_types["A"] = {
            "kind": "OBJECT",
            "fields": {
                "self": {"type": "A",
                         "nullable": False, "requires_args": False}
            }
        }
        result = self.client._build_scalar_query("A")
        self.assertNotIn("self {", result)

    @patch("main.AdaptiveClient.refresh_schema")
    @patch("main.AdaptiveClient._execute_request")
    def test_fetch_by_id_success(self, mock_exec, mock_refresh):
        """Test successful fetch by ID with valid schema and response."""
        self.client._schema_types["Book"] = {
            "kind": "OBJECT",
            "fields": {
                "id": {"type": "ID", "nullable": True, "requires_args": False},
                "title": {"type": "String",
                          "nullable": True, "requires_args": False}
            }
        }

        mock_exec.return_value = {
            "data": {"book": {"id": "1", "title": "GraphQL"}}
        }

        result = self.client.fetch_by_id("Book", 1)
        self.assertEqual(result["title"], "GraphQL")

    @patch("main.AdaptiveClient._execute_request")
    def test_run_raw_uses_json_sorted_keys(self, mock_exec):
        """Test raw query execution returns valid results."""
        mock_exec.return_value = {"data": {"ping": "pong"}}
        query = "query { ping }"
        result = self.client.run_raw(query, {})
        self.assertEqual(result["data"]["ping"], "pong")


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/079f813a-dfe3-484f-8432-093ece915c7f

#### Model code:

```python
import requests
import logging
import json
import time
import hashlib
from typing import Dict, Any

class GraphQLAdaptiveError(Exception):
    pass

class AdaptiveClient:
    introspection_query = """ … trimmed … """

    def __init__(self, endpoint: str, headers: Dict[str, str] = None, refresh_interval_seconds: int = 0):
        if not endpoint.startswith("https://"):
            raise ValueError("Only HTTPS endpoints are supported.")        #<Issue> spec: *must* raise **GraphQLAdaptiveError**
        self.endpoint = endpoint
        self.headers = headers or {}
        self.refresh_interval_seconds = refresh_interval_seconds
        self.last_schema_refresh = 0
        self.schema = None
        self.schema_hash = None
        self.session = requests.Session()
        self.session.headers.update({'Content-Type': 'application/json'})
        self.session.headers.update(self.headers)

    def refresh_schema(self, force: bool = False) -> None:
        current_time = time.time()
        if (force or (current_time - self.last_schema_refresh > self.refresh_interval_seconds)) and ("ADAPTIVE_GQL_OFFLINE" not in environ):  # <Issue> uses `environ` but never imported `os`
            try:
                response = self.session.post(
                    self.endpoint,
                    json={"query": self.introspection_query},
                    timeout=5                                         # <Issue> fixed 5 s timeout ignores --retry/timeout spec
                )
                data = response.json()
                self.schema = data['data']                           # <Issue> never computes SHA-256 or detects changes
            except Exception as e:
                logging.error(e)                                     # <Issue> logging at import-time banned; **must** log only inside public calls

```